In [17]:
import pandas as pd

# Укажите путь к файлу Excel
file_path = "Dop.xlsx"

# Загрузите файл Excel в DataFrame, указав, что первая строка содержит заголовки столбцов
df_test = pd.read_excel(file_path, header=0)

In [ ]:
%%time
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Создаем веб-драйвер Selenium (возможно, вам потребуется указать путь к исполняемому файлу вашего веб-драйвера)
driver = webdriver.Chrome()  # или webdriver.Firefox() если предпочитаете Firefox

# URL страницы

def test1(url):
    driver.get(url)

    # Ждем, пока страница загрузится (подстраивайте время по необходимости)
    wait = WebDriverWait(driver, 30)  # Увеличьте таймаут до 30 секунд или больше, если необходимо
    button = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'btn-primary')))

    # Создаем объект BeautifulSoup из HTML-содержимого страницы
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Находим все строки таблицы
    rows = soup.find_all('tr')

    # Переменная для хранения результата
    result = "НЕТ"
    text1 = ""

    # Проходим по каждой строке и проверяем текст в первой ячейке
    for row in rows:
        columns = row.find_all('td')
        if len(columns) >= 1:
            first_column_text = columns[0].get_text().strip()
            if first_column_text == "Разрешения первой категории (Лицензии)":
                # Находим атрибут onclick внутри текущей строки, если он есть
                onclick_attr = row.find('button', {'class': 'btn btn-primary btn-sm'}).get('onclick', None)
                if onclick_attr:
                    # Выполняем JavaScript-код из атрибута onclick, чтобы симулировать нажатие кнопки
                    driver.execute_script(onclick_attr)
                    result = "ДА"
                    time.sleep(1)
                break  # Выходим из цикла, так как мы нашли текст и либо нажали кнопку, либо она отсутствует
    # Если результат "ДА", то продолжаем парсить таблицу внутри модального окна
    if result == "ДА":
        # Ждем, пока появится модальное окно (подстраивайте время по необходимости)
        wait = WebDriverWait(driver, 10)
        modal_body = wait.until(EC.presence_of_element_located((By.ID, 'ModalShowFilesBody')))

        # Теперь вы можете получить HTML-содержимое модального окна
        modal_html = modal_body.get_attribute('innerHTML')

        # Создаем объект BeautifulSoup из HTML-содержимого модального окна
        modal_soup = BeautifulSoup(modal_html, 'html.parser')

        # Находим таблицу внутри модального окна по вашим критериям (например, по классу)
        table = modal_soup.find('table', {'class': 'table table-bordered'})

        # Обрабатывайте и извлекайте данные из таблицы, как вам необходимо
        if table:
            rows = table.find_all('tr')
            for row in rows:
                columns = row.find_all('td')
                for column in columns:
                    text1 += column.get_text()
                    
    return result, text1  # Return the result and extracted text

# Sample DataFrame
df = df_test

# Применить функцию к каждой строке и создать новую колонку "Result" и "DictFull"
df[['Result', 'DictFull']] = df['Links'].apply(lambda x: pd.Series(test1(x)))

print(df)

# Закройте веб-драйвер Selenium по завершении
driver.quit()

In [20]:
df.to_csv("DOP_full.csv", index=False, sep=';')